In [ ]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from dotenv import load_dotenv, find_dotenv

from langchain.tools import E2BDataAnalysisTool
import os
from langchain.agents import AgentExecutor
from langchain.agents import tool, initialize_agent, AgentType, Tool
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_anthropic.experimental import ChatAnthropicTools
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
e2b_data_analysis_tool = E2BDataAnalysisTool(api_key=os.getenv("E2B_API_KEY"))


@tool
def install_python_package_to_e2b(package_name: str):
    """
    Installs a python package to the E2B environment. This is useful for when you need to install a package that is not already installed in the E2B environment.
    """
    try:
        e2b_data_analysis_tool.install_python_packages(package_name)
        return f"Successfully installed {package_name} to the E2B environment."
    except Exception as e:
        return f"Failed to install {package_name} to the E2B environment. Error: {e}"
    

tools = [e2b_data_analysis_tool, install_python_package_to_e2b]

# OPENAI Agent

In [ ]:


# gpt-4-0125-preview
# gpt-3.5-turbo-0125
# gpt-3.5-turbo-instruct

# Create a chat prompt template
llm = ChatOpenAI(model="gpt-3.5-turbo-instruct", temperature=0,
                 openai_api_key=os.getenv("OPENAI_API_KEY"))

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful air taffic control operator, that can also use tools to answer questions and perform tasks",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [ ]:
agent_executor.invoke(
    {"input": question})

# ReACT Anthropic Agent

In [ ]:
# Artifacts are charts created by matplotlib when `plt.show()` is called
def save_artifact(artifact):
    print("New matplotlib chart generated:", artifact.name)
    # Download the artifact as `bytes` and leave it up to the user to display them (on frontend, for example)
    file = artifact.download()
    basename = os.path.basename(artifact.name)

    # Save the chart to the `charts` directory
    with open(f"./charts/{basename}", "wb") as f:
        f.write(file)


e2b_data_analysis_tool = E2BDataAnalysisTool(
    api_key=os.getenv("E2B_API_KEY"),
    on_stdout=lambda stdout: print("stdout:", stdout),
    on_stderr=lambda stderr: print("stderr:", stderr),
    on_artifact=save_artifact,
)

tools = [e2b_data_analysis_tool.as_tool(), install_python_package_to_e2b]



# Choose the LLM to use
#llm = ChatOpenAI(model='gpt-4-0125-preview')
llm = ChatAnthropicTools(model='claude-3-sonnet-20240229',anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))

#llm = ChatOpenAI()
# Get the prompt to use - you can modify this!
#prompt = hub.pull("hwchase17/react")
# Construct the ReAct agent
agent = initialize_agent(tools=tools, llm=llm, verbose=True,
                         agent=AgentType.OPENAI_FUNCTIONS)
# Create an agent executor by passing in the agent and tools
#agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent.run(question)
#agent_executor.invoke({"input": question})

In [ ]:
from langchain.agents import create_react_agent
from langchain_community.chat_models import ChatAnthropic
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

tools = [PythonREPLTool()]

instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
llm = ChatAnthropic(anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"))
llm = ChatOpenAI(model="gpt-4-0125-preview")
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": question})